# probabilistyczny:

In [7]:
try:
    import numpy as np
except ImportError:
    print("Sorry, this example requires Numpy installed !")
    raise

from easyAI import TwoPlayerGame
import random
from IPython.display import clear_output


class ConnectFour(TwoPlayerGame):
    """
    The game of Connect Four, as described here:
    http://en.wikipedia.org/wiki/Connect_Four
    """

    def __init__(self, players, board=None):
        self.players = players
        self.board = (
            board
            if (board is not None)
            else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = 1  # player 1 starts.

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        # List where number defines where the ball will fall (-1 - left column; 0 - center column; 1 - right column)
        actions = [-1, 0, 1]
        action = actions[random.randint(0, len(actions) - 1)]
        # Checking if column index is different than 0 or 6 (edges)
        if action == -1 and column != 0 or action == 1 and column != 6:
            column += action
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        # print(
        #     "\n"
        #     + "\n".join(
        #         ["0 1 2 3 4 5 6", 13 * "-"]
        #         + [
        #             " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
        #             for j in range(6)
        #         ]
        #     )
        # )
        pass

    def lose(self):
        return find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0


def find_four(board, current_player):
    """
    Returns True iff the player has connected  4 (or more)
    This is much faster if written in C or Cython
    """
    for pos, direction in POS_DIR:
        streak = 0
        while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
            if board[pos[0], pos[1]] == current_player:
                streak += 1
                if streak == 4:
                    return True
            else:
                streak = 0
            pos = pos + direction
    return False


POS_DIR = np.array(
    [[[i, 0], [0, 1]] for i in range(6)]
    + [[[0, i], [1, 0]] for i in range(7)]
    + [[[i, 0], [1, 1]] for i in range(1, 3)]
    + [[[0, i], [1, 1]] for i in range(4)]
    + [[[i, 6], [1, -1]] for i in range(1, 3)]
    + [[[0, i], [1, -1]] for i in range(3, 7)]
)



if __name__ == "__main__":
    # LET'S PLAY !

    from easyAI import AI_Player, Negamax, SSS
    
    player1wins = 0
    player2wins = 0
    
    for i in range(20):

        ai_algo_neg = Negamax(7)
        ai_algo_sss = Negamax(7)
        game = ConnectFour([AI_Player(ai_algo_neg), AI_Player(ai_algo_sss)])
        game.play()
        if game.lose():
            print("Player %d wins." % (game.opponent_index))
            if game.opponent_index == 1:
                player1wins += 1     
            else: player2wins +=1    
        else:
            print("Looks like we have a draw.")
        clear_output() 
     
    print("wygrane gracza 1 : " + str(player1wins))
    print("wygrane gracza 2 : " + str(player2wins))

wygrane gracza 1 : 10
wygrane gracza 2 : 10



# deterministyczny:

In [1]:
try:
    import numpy as np
except ImportError:
    print("Sorry, this example requires Numpy installed !")
    raise

from easyAI import TwoPlayerGame


class ConnectFour(TwoPlayerGame):
    """
    The game of Connect Four, as described here:
    http://en.wikipedia.org/wiki/Connect_Four
    """

    def __init__(self, players, board=None):
        self.players = players
        self.board = (
            board
            if (board is not None)
            else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = 1  # player 1 starts.

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        print(
            "\n"
            + "\n".join(
                ["0 1 2 3 4 5 6", 13 * "-"]
                + [
                    " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
                    for j in range(6)
                ]
            )
        )

    def lose(self):
        return find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0


def find_four(board, current_player):
    """
    Returns True iff the player has connected  4 (or more)
    This is much faster if written in C or Cython
    """
    for pos, direction in POS_DIR:
        streak = 0
        while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
            if board[pos[0], pos[1]] == current_player:
                streak += 1
                if streak == 4:
                    return True
            else:
                streak = 0
            pos = pos + direction
    return False


POS_DIR = np.array(
    [[[i, 0], [0, 1]] for i in range(6)]
    + [[[0, i], [1, 0]] for i in range(7)]
    + [[[i, 0], [1, 1]] for i in range(1, 3)]
    + [[[0, i], [1, 1]] for i in range(4)]
    + [[[i, 6], [1, -1]] for i in range(1, 3)]
    + [[[0, i], [1, -1]] for i in range(3, 7)]
)

if __name__ == "__main__":
    # LET'S PLAY !

    from easyAI import AI_Player, Negamax, SSS

    ai_algo_neg = Negamax(5)
    ai_algo_sss = SSS(5)
    game = ConnectFour([AI_Player(ai_algo_neg), AI_Player(ai_algo_sss)])
    game.play()
    if game.lose():
        print("Player %d wins." % (game.opponent_index))
    else:
        print("Looks like we have a draw.")


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Move #1: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .

Move #2: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .

Move #3: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #4: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #5: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #6: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #7: player 1 plays 1 :

0 1 2

# zaczyna gracz nr 1 dla max głebokości: 5

|                  | probabilistyczny | deterministyczny |   
|------------------|------------------|------------------|
| wygrane gracza 1 |        11        |         0        |
| wygrane gracza 2 |         9        |        20        |
|      remisy      |         0        |         0        |

# zaczyna gracz nr 2 dla max głebokości: 5

|                  | probabilistyczny | deterministyczny |   
|------------------|------------------|------------------|
| wygrane gracza 1 |         9        |        20        |
| wygrane gracza 2 |        11        |         0        |
|      remisy      |         0        |         0        |

# zaczyna gracz nr 1 dla max głebokości: 7

|                  | probabilistyczny | deterministyczny |   
|------------------|------------------|------------------|
| wygrane gracza 1 |        12        |         0        |
| wygrane gracza 2 |         8        |        20        |
|      remisy      |         0        |         0        |

# zaczyna gracz nr 2 dla max głebokości: 7

|                  | probabilistyczny | deterministyczny |   
|------------------|------------------|------------------|
| wygrane gracza 1 |        12        |         0        |
| wygrane gracza 2 |         8        |        20        |
|      remisy      |         0        |         0        |